## Description

1. Notebook holds code for:
2. generating ip ranges
3. creating connection between jdbc db(write and read)
4. two solutions for the problem

### Generator for ip ranges 
Generator is builed with python and creates ranges that can have length between 1 and 100. At the end csv file with data is created.

In [11]:
%%python
import random
import socket
import struct
socket.inet_ntoa(struct.pack('>I', random.randint(1, 0xffffffff)))
import csv  
fields = ['id', 'start', 'end']  
filename = "ips.csv"

import socket
import struct


def ip2int(addr):
    return struct.unpack("!I", socket.inet_aton(addr))[0]


def int2ip(addr):
    return socket.inet_ntoa(struct.pack("!I", addr))

def generate_ips_csv(number_of_ranges):
    with open(filename, 'w') as csvfile:  
        csvwriter = csv.writer(csvfile)  

        csvwriter.writerow(fields)  

        for i in range(number_of_ranges):
            startIp = socket.inet_ntoa(struct.pack('>I', random.randint(1, 0xffffffff)))

            randomRange = random.randint(1, 100)

            endIp = int2ip(ip2int(startIp)+randomRange)
            row = [i, startIp, endIp]
            csvwriter.writerow(row) 


In [14]:
%%python
generate_ips_csv(10)

### Scala Init

In [2]:
val x:Int = 0

Intitializing Scala interpreter ...

Spark Web UI available at http://3c68ebd29f52:4041
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1614337871675)
SparkSession available as 'spark'


x: Int = 0


### Set of properties for db connection

In [3]:
import java.util.Properties

val fileNameWithData = "ips.csv"
val url = "jdbc:mysql://mysql/test"
val driver = "com.mysql.jdbc.Driver"
val dbtable = "test"
val user = "user"
val password = "password"

val props = new Properties()
props.setProperty("user", user)
props.setProperty("password", password)
props.setProperty("driver", driver)


import java.util.Properties
fileNameWithData: String = ips.csv
url: String = jdbc:mysql://mysql/test
driver: String = com.mysql.jdbc.Driver
dbtable: String = test
user: String = user
password: String = password
props: java.util.Properties = {password=password, driver=com.mysql.jdbc.Driver, user=user}
res0: Object = null


### Read csv file and save to database

In [12]:
import org.apache.spark.sql.{DataFrame, SaveMode}
val ipAddresses = spark.read.format("csv").option("header", "true").load("ips.csv").toDF("id","start","end")
ipAddresses.write.mode(SaveMode.Overwrite).option("truncate", true).jdbc(url,dbtable, props)

<console>: 33: error: not found: value generate_ips_csv

In [5]:
val ipAddressesDataframe = spark.read.jdbc(url,dbtable, props)
ipAddressesDataframe.show(20)

+---+---------------+---------------+
| id|          start|            end|
+---+---------------+---------------+
|  0|  122.42.253.51|  122.42.253.73|
|  1|244.180.147.103|244.180.147.117|
|  2|  31.186.20.154|  31.186.20.205|
|  3|206.223.187.253| 206.223.188.34|
|  4| 146.225.225.14| 146.225.225.38|
|  5| 158.193.240.29| 158.193.240.40|
|  6|    45.9.81.100|    45.9.81.162|
|  7|  35.207.18.138|  35.207.18.157|
|  8|  199.92.142.96| 199.92.142.184|
|  9|218.144.237.124|218.144.237.138|
| 10|  43.198.90.187|   43.198.91.21|
| 11| 251.36.222.176| 251.36.222.180|
| 12|139.235.162.138|139.235.162.161|
| 13| 240.173.235.61|240.173.235.139|
| 14|102.216.137.164|102.216.137.192|
| 15| 205.220.55.115| 205.220.55.182|
| 16|  195.181.63.87| 195.181.63.138|
| 17|120.107.180.100|120.107.180.131|
| 18|  49.155.121.53|  49.155.121.85|
| 19| 117.69.238.170|  117.69.239.11|
+---+---------------+---------------+
only showing top 20 rows



ipAddressesDataframe: org.apache.spark.sql.DataFrame = [id: string, start: string ... 1 more field]


In [6]:
def ipToLong(ipAddress: String): Long = {
   ipAddress.split("\\.").reverse.zipWithIndex.map(a=>a._1.toInt*math.pow(256,a._2).toLong).sum
}

def longToIP(long: Long): String = {
   (0 until 4).map(a=>long / math.pow(256, a).floor.toInt % 256).reverse.mkString(".")
}


ipToLong: (ipAddress: String)Long
longToIP: (long: Long)String


In [7]:
val udfIpToLong = udf((s: String) => Option(s).map(str => ipToLong(str)))
val udfLongToIp = udf((s: Long) => Option(s).map(ipLong => longToIP(ipLong)))

ipAddressesDataframe
.withColumn("start_long", udfIpToLong($"start"))
.withColumn("end_long", udfIpToLong($"end")).show(3)

+---+---------------+---------------+----------+----------+
| id|          start|            end|start_long|  end_long|
+---+---------------+---------------+----------+----------+
|  0|  122.42.253.51|  122.42.253.73|2049637683|2049637705|
|  1|244.180.147.103|244.180.147.117|4105474919|4105474933|
|  2|  31.186.20.154|  31.186.20.205| 532288666| 532288717|
+---+---------------+---------------+----------+----------+
only showing top 3 rows



udfIpToLong: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$3605/0x00000008414c7840@163a183c,LongType,List(Some(class[value[0]: string])),None,true,true)
udfLongToIp: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$3609/0x00000008414d0040@59eb3dcd,StringType,List(Some(class[value[0]: bigint])),None,true,true)


### Prepare test data

In [8]:
val df = Seq(("197.203.0.0", "197.206.9.255"),
             ("197.204.0.0", "197.204.0.24"),
             ("201.233.7.160", "201.233.7.168"),
             ("201.233.7.164", "201.233.7.168"),
             ("201.233.7.167", "201.233.7.167"),
             ("203.133.1.0", "203.133.1.16")).toDF("ip_start", "ip_end")
            .withColumn("start", udfIpToLong($"ip_start"))
            .withColumn("end", udfIpToLong($"ip_end"))

val test = df.select("start","end").orderBy(asc("start")).collect().map(r=> (r(0).toString.toLong, r(1).toString.toLong))
.toMap

df: org.apache.spark.sql.DataFrame = [ip_start: string, ip_end: string ... 2 more fields]
test: scala.collection.immutable.Map[Long,Long] = Map(3318480896 -> 3318480920, 3318415360 -> 3318614527, 3387492260 -> 3387492264, 3387492256 -> 3387492264, 3387492263 -> 3387492263, 3414491392 -> 3414491408)


### Naive approach. Create set of up all ip addresses from ranges and delete those that are in other ranges

Each ip address is converted to number

In [9]:
val range  = 3318480896L to 3318480920L

def range(start: Long, end: Long): List[Long] = (start to end).toList

val udfLongToIp = udf((start: Long, end: Long) => range(start, end))

val ipAddressesPull = df.withColumn("ipAddreses", udfLongToIp($"start",$"end"))

ipAddressesPull.show(4)

+-------------+-------------+----------+----------+--------------------+
|     ip_start|       ip_end|     start|       end|          ipAddreses|
+-------------+-------------+----------+----------+--------------------+
|  197.203.0.0|197.206.9.255|3318415360|3318614527|[3318415360, 3318...|
|  197.204.0.0| 197.204.0.24|3318480896|3318480920|[3318480896, 3318...|
|201.233.7.160|201.233.7.168|3387492256|3387492264|[3387492256, 3387...|
|201.233.7.164|201.233.7.168|3387492260|3387492264|[3387492260, 3387...|
+-------------+-------------+----------+----------+--------------------+
only showing top 4 rows



range: => scala.collection.immutable.NumericRange.Inclusive[Long] <and> (start: Long, end: Long)List[Long] = NumericRange 3318480896 to 3318480920
range: => scala.collection.immutable.NumericRange.Inclusive[Long] <and> (start: Long, end: Long)List[Long]
udfLongToIp: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$3957/0x0000000841646040@1a650640,ArrayType(LongType,false),List(Some(class[value[0]: bigint]), Some(class[value[0]: bigint])),None,true,true)
ipAddressesPull: org.apache.spark.sql.DataFrame = [ip_start: string, ip_end: string ... 3 more fields]


In [10]:
val windowSpec  = Window.orderBy("ipLong")


val mutuallyExclusive = ipAddressesPull
.withColumn("ipLong",explode_outer($"ipAddreses"))
.select("ipLong")
.groupBy($"ipLong")
.count
.where($"count"===1)
.withColumn("mutuallyExclusiveIpAddresses",udfLongToIp($"ipLong"))
.select("mutuallyExclusiveIpAddresses").show(5)


<console>: 29: error: not found: value Window

In [18]:
import org.apache.spark.sql.Row
import java.time.LocalDate
import spark.implicits._


def findIntervals(arr: Seq[(Long, Long)]): scala.collection.mutable.Map[String,String] = {
    var m = scala.collection.mutable.Map.empty[String,String]
    var s = -10000L
    var max_ = -100000L
    for((start,end)<- arr){

        if (start > max_){ 
            if(s>0L && s<max_) m += (longToIP(s+1L) -> longToIP(max_)) 
            max_ = end
            s = start
        }
        else{
            if(start > s) m += (longToIP(s) -> longToIP((start-1L).toLong)) 
            if(end <=max_) s=end
            if(end > max_ || start == s){
                s = max_ + 1 
                max_ = end
            }
        }
    }
    if (max_ != -100000) m += (longToIP(s) -> longToIP(max_))
        
    println(m)
    m
}

import org.apache.spark.sql.Row
import java.time.LocalDate
import spark.implicits._
findIntervals: (arr: Seq[(Long, Long)])scala.collection.mutable.Map[String,String]


In [19]:
val df = Seq(("197.203.0.0", "197.206.9.255"),
             ("197.204.0.0", "197.204.0.24"),
             ("201.233.7.160", "201.233.7.168"),
             ("201.233.7.164", "201.233.7.168"),
             ("201.233.7.167", "201.233.7.167"),
             ("203.133.1.0", "203.133.1.16")).toDF("ip_start", "ip_end")
            .withColumn("start", udfIpToLong($"ip_start"))
            .withColumn("end", udfIpToLong($"ip_end"))

val test = df.select("start","end").collect().map(r=> (r(0).toString.toLong, r(1).toString.toLong))
.toMap

val data = test.toSeq.sortBy(_._1)

findIntervals(data)


Map(201.233.7.160 -> 201.233.7.163, 197.203.0.0 -> 197.203.255.255, 197.204.0.25 -> 197.206.9.255, 203.133.1.0 -> 203.133.1.16)


df: org.apache.spark.sql.DataFrame = [ip_start: string, ip_end: string ... 2 more fields]
test: scala.collection.immutable.Map[Long,Long] = Map(3318480896 -> 3318480920, 3318415360 -> 3318614527, 3387492260 -> 3387492264, 3387492256 -> 3387492264, 3387492263 -> 3387492263, 3414491392 -> 3414491408)
data: Seq[(Long, Long)] = Vector((3318415360,3318614527), (3318480896,3318480920), (3387492256,3387492264), (3387492260,3387492264), (3387492263,3387492263), (3414491392,3414491408))
res9: scala.collection.mutable.Map[String,String] = Map(201.233.7.160 -> 201.233.7.163, 197.203.0.0 -> 197.203.255.255, 197.204.0.25 -> 197.206.9.255, 203.133.1.0 -> 203.133.1.16)


### Whole flow at once


In [15]:
%%python
generate_ips_csv(5)

In [16]:
val ipAddresses = spark.read.format("csv").option("header", "true").load("ips.csv").toDF("id","start","end")
ipAddresses.write.mode(SaveMode.Overwrite).option("truncate", true).jdbc(url,dbtable, props)

ipAddresses: org.apache.spark.sql.DataFrame = [id: string, start: string ... 1 more field]


In [23]:
val ipAddresses = spark.read.format("csv").option("header", "true").load("ips.csv").toDF("id","start","end")
ipAddresses.write.mode(SaveMode.Overwrite).option("truncate", true).jdbc(url,dbtable, props)

val df = ipAddressesDataframe
.withColumn("start_long", udfIpToLong($"start"))
.withColumn("end_long", udfIpToLong($"end"))

val test = df.select("start_long","end_long").collect().map(r=> (r(0).toString.toLong, r(1).toString.toLong))
.toMap

val data = test.toSeq.sortBy(_._1)

findIntervals(data)

Map(51.163.31.182 -> 51.163.32.16, 95.197.236.116 -> 95.197.236.172, 21.32.134.11 -> 21.32.134.24, 175.130.32.191 -> 175.130.32.249, 218.74.66.218 -> 218.74.67.29, 248.117.102.243 -> 248.117.103.64, 128.143.211.229 -> 128.143.211.249, 168.184.213.201 -> 168.184.214.34, 82.15.212.52 -> 82.15.212.135, 67.241.152.1 -> 67.241.152.16)


ipAddresses: org.apache.spark.sql.DataFrame = [id: string, start: string ... 1 more field]
df: org.apache.spark.sql.DataFrame = [id: string, start: string ... 3 more fields]
test: scala.collection.immutable.Map[Long,Long] = Map(2944540862 -> 2944540921, 354453002 -> 354453016, 4168443635 -> 4168443712, 3662299865 -> 3662299933, 1139906560 -> 1139906576, 2156909540 -> 2156909561, 1376769075 -> 1376769159, 866328501 -> 866328592, 2830685640 -> 2830685730, 1606806643 -> 1606806700)
data: Seq[(Long, Long)] = Vector((354453002,354453016), (866328501,866328592), (1139906560,1139906576), (1376769075,1376769159), (1606806643,1606806700), (2156909540,2156909561), (2830685640,2830685730), (2944540862,2944540921), (3662299865,3662299933), (4168443635,4168443712))
res12: scala.collection.mutable.Ma...
